In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input/celeba-dataset/list_attr_celeba.csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

To read and check all the data in provided dataset

In [ ]:
import pandas as pd
import numpy as np 

In [ ]:
d1= pd.read_csv('/kaggle/input/celeba-dataset/list_landmarks_align_celeba.csv')
d2= pd.read_csv('/kaggle/input/celeba-dataset/list_eval_partition.csv')
d3= pd.read_csv('/kaggle/input/celeba-dataset/list_attr_celeba.csv')
d4= pd.read_csv('/kaggle/input/celeba-dataset/list_bbox_celeba.csv')



In [ ]:

path_dir= r'../input/celeba-dataset/img_align_celeba/img_align_celeba'


In [ ]:
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
import random

In [ ]:
for i in range(5):
    img=mpimg.imread('../input/celeba-dataset/img_align_celeba/img_align_celeba/00000'+ str(i+1)+'.jpg')
    ax=plt.subplot(1,5,i+1)
    plt.imshow(img)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

dataset = pd.read_csv('/kaggle/input/celeba-dataset/list_attr_celeba.csv')
path_dir= r'../input/celeba-dataset/img_align_celeba/img_align_celeba'

In [ ]:
from PIL import Image

im = Image.open('../input/celeba-dataset/img_align_celeba/img_align_celeba/000001.jpg')
width, height = im.size

In [ ]:
print(width, height)

In [ ]:
from PIL import Image

im = Image.open('../input/celeba-dataset/img_align_celeba/img_align_celeba/000030.jpg')
width1, height = im.size

In [ ]:
print(width, height)

In [ ]:
d1.head()

In [ ]:
d1.shape

In [ ]:
d2.head()

In [ ]:
d2.shape

In [ ]:
d3.head()

In [ ]:
d3.shape

In [ ]:
d4.head()

In [ ]:
d4.shape

In [ ]:
d3.corr()

In [ ]:
import seaborn as sns
sns.boxplot(x=d3['Arched_Eyebrows'])

In [ ]:
d3["Arched_Eyebrows"].replace({-1: 0, 1: 1}, inplace=True)

d3["Arched_Eyebrows"].head()

In [ ]:
d3["Arched_Eyebrows"].value_counts()

In [ ]:
d=pd.DataFrame(dataset.iloc[:150000])
#d=pd.DataFrame(dataset)

In [ ]:
#cols = list(dataset.columns[1:])
#cols= d['Arched_Eyebrows']

train_df, val_df = train_test_split(d, test_size=.2)



train_datagen = ImageDataGenerator(
            rescale=1./255,
    width_shift_range=.18,
    height_shift_range=.18,
            shear_range=0.5,
            zoom_range=0.6,
     horizontal_flip=True,
fill_mode= 'nearest')
val_datagen = ImageDataGenerator(rescale=1./255)
train_data = train_datagen.flow_from_dataframe(
            dataframe=train_df,
            directory=path_dir,
            x_col="image_id",
            y_col="Arched_Eyebrows",
            target_size=(170, 170),
            batch_size=52,
            class_mode='raw')

val_data = val_datagen.flow_from_dataframe(
            dataframe=val_df,
            directory=path_dir,
            x_col="image_id",
            y_col="Arched_Eyebrows",
            target_size=(170, 170),
            batch_size=52,
            class_mode='raw')


In [ ]:
from sklearn.preprocessing import LabelBinarizer
from keras.utils import to_categorical
label_binarizer = LabelBinarizer()


In [ ]:
d['Arched_Eyebrows'] = label_binarizer.fit_transform(d['Arched_Eyebrows'])
d['Arched_Eyebrows'] = to_categorical(d['Arched_Eyebrows'])


In [ ]:
from keras.applications.nasnet import NASNetLarge
from keras.models import Model
from tensorflow.keras import layers

In [ ]:
resnet=NASNetLarge(include_top=False,weights='imagenet',input_tensor=layers.Input(shape=(170,170, 3)))
for layer in resnet.layers:
    layer.trainable = False #true

resnet.summary()

In [ ]:
from tensorflow import keras
model = keras.Sequential(
    [keras.Input(shape=(170, 170, 3)),
     resnet,
     #keras.layers.MaxPooling2D(pool_size=(2,2)),
     keras.layers.Dense(780, activation="relu"),#780
     keras.layers.Dense(512, activation="relu"),
      
     keras.layers.Dropout(0.2),
     keras.layers.Flatten(),
     keras.layers.Dense(units=1,activation="sigmoid")
     ]
)

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(loss='binary_crossentropy',
              optimizer= Adam(lr=1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(
      train_data,
      steps_per_epoch=100,  #200
      epochs=5,
      verbose=1,
      validation_data = val_data,
      validation_steps=100)